In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('fitness_exercises_500.csv')

In [3]:
df.head()

,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instruction
0,Push-Up Hold,mobility,dip belt,lower,stretch,"glutes, quads, hamstrings",Push-Up Hold: Setup — Use dip belt. Focus on s...
1,Explosive Lateral Raise Pulse,cardio,barbells,full body,hold,"back, chest, legs",Explosive Lateral Raise Pulse: Setup — Use bar...
2,Rotational Jumping Jack Iso,warm-up,barbells,upper,push,"biceps, forearms, chest",Rotational Jumping Jack Iso: Setup — Use barbe...
3,Wide-Grip Running,strength,kettlebells,upper,pull,"deltoids, triceps, forearms",Wide-Grip Running: Setup — Use kettlebells. Fo...
4,Decline Dips,cardio,barbells,core,stretch,"abs, lower back, obliques",Decline Dips: Setup — Use barbells. Focus on s...


## Indexing using minsearch

In [4]:
import minsearch

In [5]:
df.columns

Index(['exercise_name', 'type_of_activity', 'type_of_equipment', 'body_part',
       'type', 'muscle_groups_activated', 'instruction'],
      dtype='object')

In [6]:
documents = df.to_dict(orient='records')

In [7]:
index = minsearch.Index(
    text_fields=['exercise_name', 'type_of_activity', 'type_of_equipment', 'body_part',
       'type', 'muscle_groups_activated', 'instruction'],
    keyword_fields=[]
)

In [8]:
index.fit(documents)

In [9]:
query = 'give me leg exercises for hamstrings'

## Implementing RAG Flow

In [10]:
from google import genai

In [11]:
client = genai.Client()

In [19]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        boost_dict=boost,
        filter_dict={},
        num_results=10
    )

    return results

In [20]:
documents[0]

{'exercise_name': 'Push-Up Hold',
 'type_of_activity': 'mobility',
 'type_of_equipment': 'dip belt',
 'body_part': 'lower',
 'type': 'stretch',
 'muscle_groups_activated': 'glutes, quads, hamstrings',
 'instruction': 'Push-Up Hold: Setup — Use dip belt. Focus on stretch movement for lower. Engage glutes, quads, hamstrings. Control breathing.'}

In [31]:
#We want the LLM to put the document content in the context of the answer
prompt_template = """
    You're a professional fitness assistant. Answer the QUESTION based only on the CONTEXT provided from the exercise & fitness database.  
    
    - Use only the facts from the CONTEXT when answering the QUESTION.  
    - If the CONTEXT does not contain the answer, respond with: NONE.  
    - Keep your answer clear, concise, and detail with instruction for fitness use.  
    
    QUESTION: {question}  
    
    CONTEXT:  
    {context}  
""".strip()

entry_template = """
exercise_name: {exercise_name}',
type_of_activity: {type_of_activity},
type_of_equipment: {type_of_equipment},
body_part: {body_part},
type: {type},
muscle_groups_activated: {muscle_groups_activated},
instruction: {instruction}
""".strip()

def build_prompt(query, search_results):    
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [32]:
def llm(prompt):
    response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt
    )

    return response.text

In [33]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [34]:
query = 'I want some core exercises that also help my back'

answer = rag(query)
print(answer)

Here are several core exercises that also help your back:

*   **Jump Rope**: Setup — Use treadmill. Focus on pull movement for core. Engage lower back, abs, obliques. Control breathing.
*   **Plank**: Setup — Use assisted machine. Focus on push movement for core. Engage abs, lower back, obliques. Control breathing.
*   **Rowing Hold**: Setup — Use kettlebells. Focus on pull movement for core. Engage abs, obliques, lower back. Control breathing.
*   **Lateral Raise**: Setup — Use dumbbells. Focus on stretch movement for core. Engage abs, obliques, lower back. Control breathing.
*   **Plank Twist**: Setup — Use treadmill. Focus on hold movement for core. Engage abs, lower back, obliques. Control breathing.
*   **Calf Raise**: Setup — Use rower. Focus on push movement for core. Engage abs, obliques, lower back. Control breathing.
*   **Squat**: Setup — Use assisted machine. Focus on stretch movement for core. Engage abs, lower back, obliques. Control breathing.
*   **Plank Reach**: Setup